### Curation Layer

#### Importing libraries

In [12]:
import import_ipynb
from pyspark.sql.functions import col

#### Setting configs and path

In [13]:
from Projects.SHIMANO_CASE_STUDY.raw_layer.file_to_raw import read_input_datasets

#### Importing Dataset

In [10]:
brands_df,categories_df,customer_df,order_items_df,orders_df,products_df,staffs_df,stocks_df,stores_df = read_input_datasets()